In [3]:
from pyomo.environ import *
from pyomo.dae import *
from pyomo.dae.simulator import Simulator

In [22]:
# Define conditions
#  A <-> B

Ca0 = 0.3    #kmol/m3
Cb0 = 0.0    #kmol/m3

F = 0.0025   #m3/min
rho = 780    #kg/m3    
Cp = 3.25    #kJ/kgK
EaA = 41570  #Kj/kmol
EaB = 45727  #Kj/kmol
DH = 4157    #Kj/kmol
k0A = 50000  #1/min
k0B = 100000 #1/min
R = 8.314    #Kj/kmolK
V = 0.2      #m3
T0 = 300     #K

def cstr(Ca0 = 0.3, Cb0 = 0.0, T0 = 300.0):
    
    #Define model
    m = ConcreteModel()
    
    # model set    
    m.t = ContinuousSet(bounds = (0, 600)) 
    
    # Model variables
    
    m.Ca = Var(m.t, initialize = 0.3, bounds=(0.0, 0.3), domain = NonNegativeReals)
    m.Cb = Var(m.t, initialize = 0.0, bounds=(0.0, None), domain = NonNegativeReals) 
    m.Temp = Var(m.t, initialize = 300, bounds=(300, None), domain = NonNegativeReals)
    m.Q = Var(m.t, bounds = (0, 5000), domain=NonNegativeReals)
    
    m.dCa = DerivativeVar(m.Ca, wrt = m.t) 
    m.dCb = DerivativeVar(m.Cb, wrt = m.t) 
    m.dTemp = DerivativeVar(m.Temp, wrt = m.t) 
    
    #Setting initial conditions
    
    m.Ca[0] = Ca0 
    m.Cb[0] = Cb0 
    m.Temp[0] = T0 
    
    #Rate
    
    rA = lambda m, t: (k0A * exp(-Ea / (R*m.Temp[t]))) * m.Ca[t] * V 
    rB = lambda m, t: (k0B * exp(-Eb / (R*m.Temp[t]))) * m.Cb[t] * V 
    
    #Constraints
    
    m.odeCa = Constraint(m.t, rule = lambda m, t: m.dCa[t] == (F * (Ca0-m.Ca[t]) - rA + rB) / V)
    m.odeCb = Constraint(m.t, rule = lambda m, t: m.dCb[t] == (F * (-m.Cb[t]) + rA - rB) / V)
    m.odeTemp = Constraint(m.t, rule = lambda m, t: m.dTemp[t] == (F * rho * Cp * (T0 - m.Temp[t])+ m.Q[t] - DH * (rA - rB))/(rho * Cp * V)) 
    
    # Objective
    
    return m
    

In [23]:
tsim, profiles = Simulator(cstr(), package='scipy').simulate(numpoints=100)

ERROR: Rule failed when generating expression for constraint odeCa with index
    0: AttributeError: 'function' object has no attribute 'is_expression_type'
ERROR: Constructing component 'odeCa' from data=None failed: AttributeError:
    'function' object has no attribute 'is_expression_type'


AttributeError: 'function' object has no attribute 'is_expression_type'